## Keras basics

Basic MLP(Multi Layer Perceptron) model

In [2]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

np.random.seed(7)

In [5]:
data = np.loadtxt('pima-indians-diabetes.data.txt', delimiter = ',')

In [7]:
X = data[:, 0:-1]
Y = data[:, -1]

In [10]:
X.shape

(768, 8)

In [11]:
Y.shape

(768,)

In [13]:
model = Sequential()
model.add(Dense(12, input_dim = 8, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.add(Dense(8, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [14]:
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [16]:
model.fit(X, Y, epochs = 150, batch_size = 10)

Epoch 1/150
768/768 [==============================] - 0s - loss: 0.6329 - acc: 0.6510     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.6324 - acc: 0.6510     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.6320 - acc: 0.6510     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.6319 - acc: 0.6510     

In [17]:
scores = model.evaluate(X,Y)

 32/768 [>.............................] - ETA: 0s('acc', 69.791666666666657)


In [22]:
model.metrics_names[1], scores[1]*100    # printing the accuracy of the prediction

('acc', 69.791666666666657)

10-fold cross-validation with the model above

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

np.random.seed(7)

In [24]:
def create_model():
    model = Sequential()
    model.add((Dense(12, input_dim = 8, kernel_initializer = 'he_normal', activation = 'relu')))
    model.add((Dense(8, kernel_initializer = 'he_normal', activation = 'relu')))
    model.add(Dense(1, kernel_initializer = 'he_normal', activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [26]:
data = np.loadtxt('pima-indians-diabetes.data.txt', delimiter = ',')
X = data[:, :-1]
Y  = data[:, -1]

In [28]:
model = KerasClassifier(build_fn = create_model, epochs = 150, batch_size = 10, verbose = 0)

In [30]:
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 7)

In [31]:
results = cross_val_score(model, X, Y, cv = kfold)

In [32]:
results

array([ 0.68831169,  0.64935065,  0.76623376,  0.64935065,  0.76623377,
        0.68831169,  0.64935065,  0.35064936,  0.72368422,  0.68421053])

In [33]:
results.mean()

0.6615686970739354

## LSTM with Keras

- Time series (Airline passengers data)

In [36]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

np.random.seed(7)

In [50]:
df = pd.read_csv('international-airline-passengers.csv', usecols = [1], engine='python', skipfooter=3)

In [52]:
data = df.values.astype('float32')
data.shape

(144, 1)

In [77]:
scaler = MinMaxScaler(feature_range = (0,1))
data = scaler.fit_transform(data)
data

array([[ 0.01544401],
       [ 0.02702703],
       [ 0.05405405],
       [ 0.04826255],
       [ 0.03281853],
       [ 0.05984557],
       [ 0.08494207],
       [ 0.08494207],
       [ 0.06177607],
       [ 0.02895753],
       [ 0.        ],
       [ 0.02702703],
       [ 0.02123553],
       [ 0.04247104],
       [ 0.07142857],
       [ 0.05984557],
       [ 0.04054055],
       [ 0.08687258],
       [ 0.12741312],
       [ 0.12741312],
       [ 0.10424709],
       [ 0.05598456],
       [ 0.01930502],
       [ 0.06949806],
       [ 0.07915059],
       [ 0.08880308],
       [ 0.14285713],
       [ 0.11389962],
       [ 0.13127413],
       [ 0.14285713],
       [ 0.18339768],
       [ 0.18339768],
       [ 0.15444016],
       [ 0.11196911],
       [ 0.08108109],
       [ 0.1196911 ],
       [ 0.12934363],
       [ 0.14671814],
       [ 0.17181468],
       [ 0.14864865],
       [ 0.15250966],
       [ 0.22007722],
       [ 0.24324325],
       [ 0.26640925],
       [ 0.2027027 ],
       [ 0

In [56]:
train_size = int(len(data) * 0.67)
train_data , test_data = data[:train_size, :], data[train_size:, :]

In [37]:
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back -1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [58]:
look_back = 1
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

In [60]:
X_train.shape

(94, 1)

In [61]:
X_test.shape

(46, 1)

In [62]:
Y_train.shape

(94,)

In [63]:
Y_test.shape

(46,)

In [78]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape)
print(X_test.shape)

(94, 1, 1)
(46, 1, 1)


In [67]:
model = Sequential()
model.add(LSTM(4, input_shape = (None, look_back)))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(X_train, Y_train, epochs = 100, batch_size = 1, verbose = 2)

Epoch 1/100
0s - loss: 0.0440
Epoch 2/100
0s - loss: 0.0214
Epoch 3/100
0s - loss: 0.0151
Epoch 4/100
0s - loss: 0.0135
Epoch 5/100
0s - loss: 0.0126
Epoch 6/100
0s - loss: 0.0116
Epoch 7/100
0s - loss: 0.0107
Epoch 8/100
0s - loss: 0.0097
Epoch 9/100
0s - loss: 0.0087
Epoch 10/100
0s - loss: 0.0078
Epoch 11/100
0s - loss: 0.0069
Epoch 12/100
0s - loss: 0.0061
Epoch 13/100
0s - loss: 0.0053
Epoch 14/100
0s - loss: 0.0046
Epoch 15/100
0s - loss: 0.0040
Epoch 16/100
0s - loss: 0.0035
Epoch 17/100
0s - loss: 0.0031
Epoch 18/100
0s - loss: 0.0028
Epoch 19/100
0s - loss: 0.0025
Epoch 20/100
0s - loss: 0.0024
Epoch 21/100
0s - loss: 0.0022
Epoch 22/100
0s - loss: 0.0021
Epoch 23/100
0s - loss: 0.0021
Epoch 24/100
0s - loss: 0.0020
Epoch 25/100
0s - loss: 0.0020
Epoch 26/100
0s - loss: 0.0020
Epoch 27/100
0s - loss: 0.0021
Epoch 28/100
0s - loss: 0.0021
Epoch 29/100
0s - loss: 0.0020
Epoch 30/100
0s - loss: 0.0020
Epoch 31/100
0s - loss: 0.0020
Epoch 32/100
0s - loss: 0.0021
Epoch 33/100
0s -

In [71]:
train_prediction = scaler.inverse_transform(model.predict(X_train))
test_prediction = scaler.inverse_transform(model.predict(X_test))
Y_train = scaler.inverse_transform([Y_train])
Y_test = scaler.inverse_transform([Y_test])

In [73]:
train_score = math.sqrt(mean_squared_error(Y_train[0], train_prediction[:, 0]))
print('Train Score: {} RMSE'.format(train_score))

Train Score: 23.64529144552407 RMSE


In [74]:
test_score = math.sqrt(mean_squared_error(Y_test[0], test_prediction[:, 0]))
print('Test Score: {} RMSE'.format(test_score))

Test Score: 52.926745085080775 RMSE


## 실습 4-4-1. LSTM을 활용한 IMDB 영화 리뷰 감성분석 (1)
- 학습 데이터와 검증 데이터를 각각 2000개씩 배정한다
- 검증 데이터에 대한 정확도(accuracy)를 계산해 본다

In [101]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

np.random.seed(7)

In [117]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

In [103]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_len)[:2000, :]
X_test = sequence.pad_sequences(X_test, maxlen = max_len)[:2000, :]

In [104]:
y_train = y_train[:2000]
y_test = y_test[:2000]

In [105]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))

In [106]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [109]:
model.fit(X_train, y_train, epochs = 3, batch_size = 64)
scores = model.evaluate(X_test, y_test, verbose = 0)

Epoch 1/3
1000/1000 [==============================] - 15s - loss: 0.1894 - acc: 0.9470     

In [110]:
print(scores[1])

0.76


## 실습 4-4-2. LSTM을 활용한 IMDB 영화 리뷰 감성분석 (2)
- 학습 데이터와 검증 데이터 전체를 활용한다
- 검증 데이터에 대한 정확도(accuracy)를 계산해 본다

In [111]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

np.random.seed(7)

In [112]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

In [120]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_len)
X_test = sequence.pad_sequences(X_test, maxlen = max_len)

In [114]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))

In [115]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [116]:
model.fit(X_train, y_train, epochs = 3, batch_size = 64)

Epoch 1/3
25000/25000 [==============================] - 411s - loss: 0.4884 - acc: 0.7594     

In [121]:
scores = model.evaluate(X_test, y_test, verbose = 0)

In [122]:
print(scores[1])

0.80944
